In [1]:
import pandas as pd
import os
import yaml
import numpy as np
from copy import deepcopy
import pdb

# Pickup all excel sheets

In [2]:
SURVEY_LIST = ['config_echoes.yaml', 'config_briskee.yaml', 'config_cheetah.yaml', 'config_natc.yaml']
SURVEY_CHOICE = 2

#TODO: Memory leak in cheetah ??
#TODO: Correct first graph figure plots that are squashed along x-axis 

with open(SURVEY_LIST[SURVEY_CHOICE], 'r') as f:
    args = yaml.safe_load(f)

data_files = []
dirs = list(set([__dir if os.path.isdir(os.path.join(args['data_dir'], __dir)) else '' for __dir in os.listdir(args['data_dir'])]))
for _dir in dirs:
    for filename in os.listdir(os.path.join(args['data_dir'], _dir)):
        if args['format'] == 'rda': # scan for .rdata/.rda
            if filename.endswith(".rda") or filename.endswith(".rdata"): 
                data_files.append(os.path.join(args['data_dir'], _dir, filename))
            else:
                continue
        elif args['format'] == 'xls': # scan for .xls/.xlsx
            if filename.endswith(".xls") or filename.endswith(".xlsx"): 
                 data_files.append(os.path.join(args['data_dir'], _dir, filename))
            else:
                continue
        elif args['format'] == 'csv': # scan for .csv
            if filename.endswith(".csv"): 
                 data_files.append(os.path.join(args['data_dir'], _dir, filename))
            else:
                continue

        else:
            print('Unsupported file format selcted')

print(data_files)
print(len(data_files))

out_dir = os.path.join(os.getcwd(), args['out_dir'])
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
else:
    print(f"{out_dir} directory: exists")
    

['/media/windowsC/SIN_Desktop/ENCHANT/wp3_datasets/cheetah_data_sharing/ITthermostat.csv', '/media/windowsC/SIN_Desktop/ENCHANT/wp3_datasets/cheetah_data_sharing/DEfridge.csv', '/media/windowsC/SIN_Desktop/ENCHANT/wp3_datasets/cheetah_data_sharing/DEthermostat.csv', '/media/windowsC/SIN_Desktop/ENCHANT/wp3_datasets/cheetah_data_sharing/ESfridge.csv', '/media/windowsC/SIN_Desktop/ENCHANT/wp3_datasets/cheetah_data_sharing/ESthermostat.csv', '/media/windowsC/SIN_Desktop/ENCHANT/wp3_datasets/cheetah_data_sharing/FRfridge.csv', '/media/windowsC/SIN_Desktop/ENCHANT/wp3_datasets/cheetah_data_sharing/FRthermostat.csv', '/media/windowsC/SIN_Desktop/ENCHANT/wp3_datasets/cheetah_data_sharing/ITfridge.csv', '/media/windowsC/SIN_Desktop/ENCHANT/wp3_datasets/cheetah_data_sharing/ITpolicy.csv', '/media/windowsC/SIN_Desktop/ENCHANT/wp3_datasets/cheetah_data_sharing/PLfridge.csv', '/media/windowsC/SIN_Desktop/ENCHANT/wp3_datasets/cheetah_data_sharing/PLheating.csv', '/media/windowsC/SIN_Desktop/ENCHANT

# Sanity checks and iterate over individual sheet

In [3]:
import subprocess
import inspect

class SurveyParser(object):
    
    default_args = {
        'separator': ',',
        'ignore_files_with_phrases': ['codebook', 'codes_labels']
    }
    
    def get_xls_df_list(self, 
                        _data_files, 
                        **kwargs):
        survey_df_dict = {}
        remove_files_with_phrases = kwargs.default_args['ignore_files_with_phrases'] if 'ignore_files_with_phrases' in kwargs.keys() else self.default_args['ignore_files_with_phrases']
        data_files = [_file for _file in _data_files if not any(phrase in _file for phrase in remove_files_with_phrases)]

        for _file in data_files:
            try:
                sheet = pd.read_excel(_file, sheet_name=0) # Access the first sheet the other are codes and questionaire interpretations
            except Exception as e:
                print('Exception caught in xls')

                _csv = '.'.join(_file.split('.')[:-1])+'.csv'
                if os.path.exists(_csv+'.0'):
                    print('File exists: Skip xls to csv conversion')
                else:
                    #pdb.set_trace()
                    # Try converting xls to csv through CLI
                    cmd = ['ssconvert', '-S', _file, _csv]
                    proc = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

                    o, e = proc.communicate()

                sheet = pd.read_csv(_csv+'.0')

            name = _file.replace(" ", "_").split('/')[-1]
            survey_df_dict[name] = sheet
            sheet = None

        return survey_df_dict

    def get_csv_df_list(self, 
                        _data_files, 
                        **kwargs):
        survey_df_dict = {}
        remove_files_with_phrases = kwargs['ignore_files_with_phrases'] if 'ignore_files_with_phrases' in kwargs.keys() else self.default_args['ignore_files_with_phrases']
        separator = kwargs['separator'] if 'separator' in kwargs.keys() else self.default_args['separator']
        data_files = [_file for _file in _data_files if not any(phrase in _file for phrase in remove_files_with_phrases)]
        
        for _file in data_files:
            sheet = pd.read_csv(_file, 
                                sep=separator,
                                encoding='unicode_escape',
                               low_memory=False)

            name = _file.replace(" ", "_").split('/')[-1]
            survey_df_dict[name] = sheet
            sheet = None

        return survey_df_dict

    def get_spss_df_list(self, 
                        _data_files, 
                        **kwargs):
        return survey_df_dict

    
def get_methods(class_arg):

    assert inspect.isclass(class_arg), 'Expecting a class as an argument'
    
    method_list = []
    # attribute is a string representing the attribute name
    for attribute in dir(class_arg):
        # Get the attribute value
        attribute_value = getattr(class_arg, attribute)
        # Check that it is callable
        if callable(attribute_value):
            # Filter all dunder (__ prefix) methods
            if attribute.startswith('__') == False:
                method_list.append(attribute)

    return method_list


def get_method_fn(method,
                  method_key,
                  methods_list,
                  methods_class):
    
    assert method in methods_list, f'Error: Improper method selected in {method_key} in config.yaml'
    fn_method = getattr(methods_class, method)
    print(f"{method_key}: {fn_method}")
    return fn_method


parser_class_inst = SurveyParser()
parser_list = get_methods(SurveyParser)
print(parser_list)
parser_fn = get_method_fn(f"get_{args['format']}_df_list",
                          args['format'],
                          parser_list,
                          SurveyParser)

survey_df_dict = parser_fn(parser_class_inst, data_files, **args)  # Example of procedure call with class instance passing

['get_csv_df_list', 'get_spss_df_list', 'get_xls_df_list']
csv: <function SurveyParser.get_csv_df_list at 0x7f8b78026b90>


In [4]:
'''TODO: Handle None dfs'''
for name, survey_df in survey_df_dict.items():
    print(f"{name}: {survey_df.shape}")
#print(survey_df)

ITthermostat.csv: (5148, 151)
DEfridge.csv: (7008, 138)
DEthermostat.csv: (6876, 178)
ESfridge.csv: (7248, 121)
ESthermostat.csv: (8832, 167)
FRfridge.csv: (17184, 130)
FRthermostat.csv: (6000, 172)
ITfridge.csv: (6640, 120)
ITpolicy.csv: (18234, 112)
PLfridge.csv: (7888, 129)
PLheating.csv: (11256, 155)
PLpolicy.csv: (18306, 112)
PLthermostat.csv: (5688, 172)
ROfridge.csv: (9504, 129)
ROthermostat.csv: (5616, 172)
SEfridge.csv: (9584, 129)
SEheating.csv: (9540, 155)
SEpolicy.csv: (18072, 112)
SEthermostat.csv: (6900, 172)
UKfridge.csv: (7856, 177)
UKheating.csv: (8376, 162)
UKpolicy.csv: (16686, 112)
UKthermostat.csv: (7584, 176)


In [5]:
class SurveyConditioning(object):
    
    def get_nan_idices(sheet):
        '''
        '''
        idx = np.argwhere(pd.isnull(sheet).to_numpy()) #2D indices
        #print(idx)
        return idx
    
    
    def no_nan(_sheet):
        '''
        '''
        sheet = _sheet.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
        #print(sheet.shape)
        return sheet

    def substitute_nan(_sheet):
        '''
        '''
        sheet = _sheet.fillna(0.0)
        #print(sheet.shape)
        return sheet

In [6]:
conditioning_class_inst = SurveyConditioning()
methods_list = get_methods(SurveyConditioning)
print(methods_list)

['get_nan_idices', 'no_nan', 'substitute_nan']


In [7]:
print(args)

{'format': 'csv', 'separator': ',', 'data_dir': '/media/windowsC/SIN_Desktop/ENCHANT/wp3_datasets/cheetah_data_sharing', 'out_dir': '../../cheetah/plots', 'surveys': {'global': {'histogram': {'enable': True, 'ROWS': 10, 'COLS': 10, 'SIZE': ['8*8', '6*8'], 'padding': 1.0}, 'conditioning': 'substitute_nan'}, 'DEfridge': {'histogram': {'ROWS': 14}}, 'DEthermostat': {'histogram': {'ROWS': 18}}, 'ESfridge': {'histogram': {'ROWS': 13}}, 'ESthermostat': {'histogram': {'ROWS': 17}}, 'FRfridge': {'histogram': {'ROWS': 13}}, 'FRthermostat': {'histogram': {'ROWS': 18}}, 'ITfridge': {'histogram': {'ROWS': 12}}, 'ITpolicy': {'histogram': {'ROWS': 12}}, 'ITthermostat': {'histogram': {'ROWS': 16}}, 'PLfridge': {'histogram': {'ROWS': 13}}, 'PLheating': {'histogram': {'ROWS': 16}}, 'PLpolicy': {'histogram': {'ROWS': 12}}, 'PLthermostat': {'histogram': {'ROWS': 18}}, 'ROfridge': {'histogram': {'ROWS': 13}}, 'ROthermostat': {'histogram': {'ROWS': 18}}, 'SEfridge': {'histogram': {'ROWS': 13}}, 'SEheating'

In [ ]:
import gc
import matplotlib.pyplot as plt

class PlottingClass:
    def build_histogram(self,
                        out_dir,
                        parsed_df,
                        settings,
                        survey_name,
                        ignore_label=0):
        #df = pd.DataFrame({'species': ['bear', 'bear', 'marsupial'], 'population': [1864, 22000, 80000]}, index=['panda', 'polar', 'koala'])
        parsed_df = parsed_df.iloc[settings.get("n_discard_columns", 1):, :]
        ignore_labels = settings.get("ignore_labels", [ignore_label])

        ROWS, COLS, SIZE = settings.get("ROWS", 0), settings.get("COLS", 0), settings.get("SIZE", (['8*8', '6*8']))
        assert not (ROWS == 0 and COLS == 0), f'{survey_name} Error: ROWS/COL value not specified in config.yaml'

        SIZE = list(map(lambda x: eval(x), SIZE))
        fig, axes = plt.subplots(figsize=SIZE, dpi=300, nrows=ROWS, ncols=COLS)
        
        padding = settings.get("padding", 3.0)
        fig.tight_layout(pad=padding)
        plt.savefig(os.path.join(out_dir,f'{survey_name}.png'), bbox_inches='tight')

        fig.clf()
        plt.close(fig)
        plt.close('all')



def build_histogram(out_dir,
                    parsed_df,
                    settings,
                    survey_name,
                    ignore_label=0):
    #df = pd.DataFrame({'species': ['bear', 'bear', 'marsupial'], 'population': [1864, 22000, 80000]}, index=['panda', 'polar', 'koala'])
    parsed_df = parsed_df.iloc[settings.get("n_discard_columns", 1):, :]
    ignore_labels = settings.get("ignore_labels", [ignore_label])

    ROWS, COLS, SIZE = settings.get("ROWS", 0), settings.get("COLS", 0), settings.get("SIZE", (['8*8', '6*8']))
    assert not (ROWS == 0 and COLS == 0), f'{survey_name} Error: ROWS/COL value not specified in config.yaml'

    SIZE = list(map(lambda x: eval(x), SIZE))
    fig, axes = plt.subplots(figsize=SIZE, dpi=300, nrows=ROWS, ncols=COLS)
    
    for idx, (name, col) in enumerate(parsed_df.items()):
        #assert all(col.map(type) == int) or all(col.map(type) == float)
        sr_count = col.value_counts()
        _ignore_labels = [label for label in ignore_labels if label in sr_count.keys()]
        if len(_ignore_labels) > 0:
            sr_count.drop(labels=_ignore_labels, inplace=True)
            #pdb.set_trace()

        i, j = idx // COLS, idx % COLS
        if i>= ROWS or j>= COLS:
            pdb.set_trace()
        #print(sr_count.keys().dtype, sr_count.values.dtype)
        ax = axes[i, j]

        if sr_count.sum() > 0:  #Plot the responses in corresponding subplot
            FIRST_TIME = True
            while FIRST_TIME:
                try:
                    ax.bar(sr_count.keys(), sr_count.values, align='center', color='#607c8e')
                    ax.plot(sr_count.keys(), sr_count.values, marker="o", linestyle="", alpha=0.8, color="b")
                    FIRST_TIME = False
                except TypeError:
                    if FIRST_TIME:
                        sr_count.index = list(map(str, list(sr_count.keys())))  # Ensure consistent type first time per plot
                        FIRST_TIME = False
                    else:
                        pdb.set_trace()
            ax.set_title(f'{name}', fontsize=7, fontweight='bold', color='blue')

            ax.xaxis.set_tick_params(labelsize=5, labelcolor='r')
            ax.set_xlabel(f'response[{sr_count.sum()}]', fontsize=5, fontweight='bold')
            ax.yaxis.set_tick_params(labelsize=5, labelcolor='r')
            ax.set_ylabel('count', fontsize=5, fontweight='bold')
            ax.grid(axis='y', alpha=0.75)
        else:  #No responses: remove corresponding subplot
            ax.remove()

    padding = settings.get("padding", 3.0)
    fig.tight_layout(pad=padding)
    plt.savefig(os.path.join(out_dir,f'{survey_name}.png'), bbox_inches='tight')

    fig.clf()
    plt.close(fig)
    plt.close('all')

try:
    survey_global_settings = args['surveys'].pop('global')
except "KeyError":
    survey_global_settings = None

def recursive_parse_settings(given_setting, modify_setting):
    for key, val in modify_setting.items():
        if isinstance(val, dict):
            recursive_parse_settings(given_setting[key], val)
        else:
            given_setting[key] = val
    return given_setting


from termcolor import colored

print(args['surveys'].keys())
for key, _val in args['surveys'].items():
    #pdb.set_trace()
    global_settings = deepcopy(survey_global_settings)
    print(colored(f'============================================ {key} ============================================', 'yellow'))
    val = recursive_parse_settings(global_settings, _val) if survey_global_settings is not None else _val
    print(f"survey_global_settings:{survey_global_settings}\n, val:{val}")
    #pdb.set_trace()
    
    method_fn = get_method_fn(val['conditioning'],
                              key,
                              methods_list,
                              SurveyConditioning)
    
    matching_df = [survey_df for file_name, survey_df in survey_df_dict.items() if key in file_name]
    assert len(matching_df) > 0, 'Error: Improper survey name in config.yaml'
    parsed_df = method_fn(matching_df[0])  # Example of procedure call without class instance passing
    print(f"before:{matching_df[0].shape}, after:{parsed_df.shape}")
    
    pltClass = PlottingClass()
    # Build a histogram of each column in df
    if val['histogram']['enable']:
        pltClass.build_histogram(out_dir, 
                        parsed_df,
                        val['histogram'], 
                        f"{key}_{val['conditioning']}")
    del pltClass, global_settings, val, method_fn, matching_df, parsed_df
    gc.collect


dict_keys(['DEfridge', 'DEthermostat', 'ESfridge', 'ESthermostat', 'FRfridge', 'FRthermostat', 'ITfridge', 'ITpolicy', 'ITthermostat', 'PLfridge', 'PLheating', 'PLpolicy', 'PLthermostat', 'ROfridge', 'ROthermostat', 'SEfridge', 'SEheating', 'SEpolicy', 'SEthermostat', 'UKfridge', 'UKheating', 'UKpolicy', 'UKthermostat'])
============================================ DEfridge ============================================
survey_global_settings:{'histogram': {'enable': True, 'ROWS': 10, 'COLS': 10, 'SIZE': ['8*8', '6*8'], 'padding': 1.0}, 'conditioning': 'substitute_nan'}
, val:{'histogram': {'enable': True, 'ROWS': 14, 'COLS': 10, 'SIZE': ['8*8', '6*8'], 'padding': 1.0}, 'conditioning': 'substitute_nan'}
DEfridge: <function SurveyConditioning.substitute_nan at 0x7f8b772899e0>
before:(7008, 138), after:(7008, 138)
============================================ DEthermostat ============================================
survey_global_settings:{'histogram': {'enable': True, 'ROWS': 10, 'COLS': 1

KeyboardInterrupt: 